In [26]:
!mkdir ../../outputs/results/suppledata

In [91]:
import pandas as pd

model_name = ["Control"] + ["Head"] + [f"Block {7-i}" for i in range(7)] + ["Full"]
layer_name = ["Stem"] + [f"Block {i+1}" for i in range(7)] + ["Head"]


In [104]:
import numpy as np

dfc = pd.DataFrame()
df = pd.read_csv(f"../../outputs/results/230308finding_type_validation_loo_seed123_epoch5.csv")
for c in ["model", "layer", "pathological findings"]:
    dfc[c] = df[c]
for mtrc in ["AUROC", "MCC", "Balanced Accuracy", "AP"]:
    buf = pd.DataFrame()
    for seed in range(5):
        if mtrc != "AP":
            df = pd.read_csv(f"../../outputs/results/230308finding_type_validation_loo_seed{seed+123}_epoch5.csv")
        else:
            df = pd.read_csv(f"../../outputs/results/230721finding_type_validation_loo_seed{seed+123}_epoch5.csv")
        buf[seed] = df[mtrc]
    dfc[mtrc + "_mean"] = buf.mean(axis=1)
    dfc[mtrc + "_std"] = buf.std(axis=1, ddof=1).apply(lambda x : x if x >= 1e-9 else np.nan)


In [96]:
import numpy as np
dft = dfc.groupby(["model", "layer"]).mean().T

var_df = pd.DataFrame()

for mtrc in ["AUROC", "MCC", "Balanced Accuracy", "AP"]:
    vars = []
    for i, m in enumerate(model_name):
        dat = [dft[(m, l)][mtrc + "_mean"] for j, l in enumerate(layer_name)]
        if len(dat) == 1:
            vars.append(np.nan)
        else:
            vars.append(np.var(dat, ddof=0))
    var_df[mtrc] = vars
var_df.index = model_name
var_df
var_df.to_csv("../../outputs/results/230721var_model_loo.csv")

In [97]:
var_df = pd.DataFrame()

for mtrc in ["AUROC", "MCC", "Balanced Accuracy", "AP"]:
    vars = []
    for i, l in enumerate(layer_name):
        dat = [dft[(m, l)][mtrc + "_mean"] for j, m in enumerate(model_name)]
        if len(dat) == 1:
            vars.append(np.nan)
        else:
            vars.append(np.var(dat, ddof=0))
    var_df[mtrc] = vars
var_df.index = layer_name
var_df.to_csv("../../outputs/results/230721var_layer_loo.csv")

In [98]:
dft[(m, l)]["AUROC_mean"]

0.8530151109329777

In [99]:
dfc.groupby(["model", "layer"]).mean().groupby(["layer"]).var(ddof=0)

,AUROC_mean,AUROC_std,MCC_mean,MCC_std,Balanced Accuracy_mean,Balanced Accuracy_std,AP_mean,AP_std
layer,,,,,,,,
-,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000e+00,NaN
Block 1,0.000330,0.000007,0.000023,6.004494e-07,0.000070,4.039317e-07,3.193178e-05,3.164828e-07
Block 2,0.000102,0.000012,0.000959,4.148527e-05,0.000237,1.130267e-06,5.533012e-05,4.662129e-06
Block 3,0.000598,0.000061,0.000375,1.384214e-04,0.000112,2.619341e-05,9.939128e-06,2.881764e-05
Block 4,0.000313,0.000031,0.000021,9.029619e-05,0.000004,1.483434e-05,4.571189e-05,4.846123e-05
Block 5,0.000435,0.000015,0.000226,1.840565e-04,0.000051,2.729995e-05,1.058500e-04,8.442753e-05
Block 6,0.000329,0.000067,0.000403,4.123176e-04,0.000084,6.545477e-05,3.698210e-04,1.097172e-04
Block 7,0.000640,0.000116,0.001336,3.962256e-04,0.000285,7.847395e-05,8.412024e-04,9.104753e-05
Head,0.000222,0.000161,0.001199,4.516876e-04,0.000180,1.249682e-04,7.743007e-04,1.319276e-04


In [100]:
model_name = ["Control"] + ["Head"] + [f"Block {7-i}" for i in range(7)] + ["Full"]
layer_name = ["Stem"] + [f"Block {i+1}" for i in range(7)] + ["Head"]
ft_list = ['Proliferation, bile duct',
 'Ground glass appearance',
 'Increased mitosis',
 'Inclusion body, intracytoplasmic',
 'Deposit, pigment',
 'Single cell necrosis',
 'Vacuolization, cytoplasmic',
 'Swelling']
ind = []
for m in model_name:
    for l in layer_name:
        for p in ft_list:
            ind.append((m,l,p))

In [105]:
dfs = []
for seed in range(5):
    df = pd.read_csv(f"../../outputs/results/230308finding_type_validation_loo_seed{seed+123}_epoch5.csv")
    df1 = pd.read_csv(f"../../outputs/results/230721finding_type_validation_loo_seed{seed+123}_epoch5.csv")
    del df["AP"]
    df["mAP"] = df1["AP"]
    df["seed"] = 123 + seed
    dfs.append(df)
df = pd.concat(dfs)
df.to_csv("../../outputs/results/suppledata/230723LOOraw.csv", index=False)

In [106]:
mean = df.groupby(["model", "layer", "pathological findings"]).mean().reindex(ind)
del mean["seed"]
mean.columns = [c + "_mean" for c in mean.columns]
std = df.groupby(["model", "layer", "pathological findings"]).std().reindex(ind)
del std["seed"]
std.columns = [c + "_std" for c in std.columns]
df = pd.concat([mean, std], axis=1)
df = df[[mtrc + "_" + st for mtrc in ["AUROC", "MCC", "Balanced Accuracy", "mAP"] for st in ["mean", "std"]]]
df.to_csv("../../outputs/results/suppledata/230723LOO.csv")

In [58]:
dfs = []
for seed in range(5):
    df = pd.read_csv(f"../../outputs/results/230704compound_validation_{123+seed}.csv")
    df1 = pd.read_csv(f"../../outputs/results/230704compound_validation_macro_{123+seed}.csv")
    df["Balanced Accuracy (Multiclass)"] = df1["Macro Balanced Accuracy"]
    df["seed"] = 123 + seed
    dfs.append(df)
df = pd.concat(dfs)
df.to_csv("../../outputs/results/suppledata/MoAraw.csv", index=False)

In [59]:
df["Balanced Accuracy (Multiclass)"]

0       0.321930
1       0.459274
2       0.336667
3       0.316281
4       0.343155
          ...   
4545    0.400000
4546    0.284444
4547    0.142857
4548    0.452593
4549    0.353333
Name: Balanced Accuracy (Multiclass), Length: 22750, dtype: float64

In [60]:
df.columns

Index(['model', 'layer', 'trial', 'fold',
       'Bacterial 70S ribosome inhibitor_Balanced Accuracy',
       'Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy',
       'DNA inhibitor_Balanced Accuracy',
       'Cyclooxygenase inhibitor_Balanced Accuracy',
       'Sulfonylurea receptor 1, Kir6.2 blocker_Balanced Accuracy',
       'Histamine H2 receptor antagonist_Balanced Accuracy',
       'Peroxisome proliferator-activated receptor alpha agonist_Balanced Accuracy',
       'Bacterial 70S ribosome inhibitor_AUROC',
       'Serotonin 2a (5-HT2a) receptor antagonist_AUROC',
       'DNA inhibitor_AUROC', 'Cyclooxygenase inhibitor_AUROC',
       'Sulfonylurea receptor 1, Kir6.2 blocker_AUROC',
       'Histamine H2 receptor antagonist_AUROC',
       'Peroxisome proliferator-activated receptor alpha agonist_AUROC',
       'Bacterial 70S ribosome inhibitor_mAP',
       'Serotonin 2a (5-HT2a) receptor antagonist_mAP', 'DNA inhibitor_mAP',
       'Cyclooxygenase inhibitor_mAP',
      

In [61]:
dft = df.groupby(["model", "layer"]).mean().T

var_df = pd.DataFrame()

for mtrc in ['Balanced Accuracy (Multiclass)']:
    vars = []
    for i, m in enumerate(model_name):
        dat = [dft[(m, l)][mtrc] for j, l in enumerate(layer_name)]
        if len(dat) == 1:
            vars.append(np.nan)
        else:
            vars.append(np.var(dat, ddof=0))
    var_df[mtrc] = vars
var_df.index = model_name
var_df.to_csv("../../outputs/results/230716var_model_MoA.csv")

In [62]:
var_df = pd.DataFrame()

for mtrc in ['Balanced Accuracy (Multiclass)']:
    vars = []
    for i, l in enumerate(layer_name):
        dat = [dft[(m, l)][mtrc] for j, m in enumerate(model_name)]
        if len(dat) == 1:
            vars.append(np.nan)
        else:
            vars.append(np.var(dat, ddof=0))
    var_df[mtrc] = vars
var_df.index = layer_name
var_df.to_csv("../../outputs/results/230716var_layer_MoA.csv")

In [63]:
df = df.groupby(["model", "layer", "seed"]).mean()
del df["trial"]
del df["fold"]

In [64]:
df

Bacterial 70S ribosome inhibitor_Balanced Accuracy  \
model                 layer   seed                                                       
Block 1               Block 1 123                                            0.547616    
                              124                                            0.546751    
                              125                                            0.540133    
                              126                                            0.553272    
                              127                                            0.541441    
...                                                                               ...    
Pathological Findings -       123                                            0.497962    
                              124                                            0.497962    
                              125                                            0.497962    
                              126                                            0.497962    
                              127                                            0.497962    

                                    Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy  \
model                 layer   seed                                                                
Block 1               Block 1 123                                            0.498267             
                              124                                            0.498415             
                              125                                            0.498556             
                              126                                            0.498713             
                              127                                            0.498260             
...                                                                               ...             
Pathological Findings -       123                                            0.500000             
                              124                                            0.500000             
                              125                                            0.500000             
                              126                                            0.500000             
                              127                                            0.500000             

                                    DNA inhibitor_Balanced Accuracy  \
model                 layer   seed                                    
Block 1               Block 1 123                          0.728237   
                              124                          0.735414   
                              125                          0.755215   
                              126                          0.743006   
                              127                          0.734605   
...                                                             ...   
Pathological Findings -       123                          0.529767   
                              124                          0.529767   
                              125                          0.529767   
                              126                          0.529767   
                              127                          0.529767   

                                    Cyclooxygenase inhibitor_Balanced Accuracy  \
model                 layer   seed                                               
Block 1               Block 1 123                                     0.658880   
                              124                                     0.662186   
                              125                                     0.678993   
                              126                                     0.673361   
                              127                                     0.656432   
...                                                                        ... 

In [65]:
ind = [("Pathological Findings", "-")]
for m in model_name:
    for l in layer_name:
        ind.append((m,l))
mean = df.groupby(["model", "layer"]).mean().reindex(ind)
mean

Bacterial 70S ribosome inhibitor_Balanced Accuracy  \
model                 layer                                                         
Pathological Findings -                                                 0.497962    
Control               Stem                                              0.517367    
                      Block 1                                           0.525001    
                      Block 2                                           0.613446    
                      Block 3                                           0.635521    
...                                                                          ...    
Full                  Block 4                                           0.772522    
                      Block 5                                           0.740049    
                      Block 6                                           0.782984    
                      Block 7                                           0.761306    
                      Head                                              0.537649    

                               Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy  \
model                 layer                                                                  
Pathological Findings -                                                 0.500000             
Control               Stem                                              0.500000             
                      Block 1                                           0.499284             
                      Block 2                                           0.591252             
                      Block 3                                           0.571628             
...                                                                          ...             
Full                  Block 4                                           0.593101             
                      Block 5                                           0.606130             
                      Block 6                                           0.596273             
                      Block 7                                           0.581364             
                      Head                                              0.500060             

                               DNA inhibitor_Balanced Accuracy  \
model                 layer                                      
Pathological Findings -                               0.529767   
Control               Stem                            0.662287   
                      Block 1                         0.753970   
                      Block 2                         0.777810   
                      Block 3                         0.803146   
...                                                        ...   
Full                  Block 4                         0.888831   
                      Block 5                         0.875185   
                      Block 6                         0.886520   
                      Block 7                         0.890122   
                      Head                            0.821786   

                               Cyclooxygenase inhibitor_Balanced Accuracy  \
model                 layer                                                 
Pathological Findings -                                          0.593824   
Control               Stem                                       0.624035   
                      Block 1                                    0.663464   
                      Block 2                                    0.711469   
                      Block 3                                    0.764898   
...                                                                   ...   
Full                  Block 4                                    0.843877   
                      Block 5                                    0.841175   
                      Block 6                                    0.857479

In [66]:
mean = df.groupby(["model", "layer"]).mean().reindex(ind)
col = list(mean.columns)
mean.columns = [c + "_mean" for c in mean.columns]
std = df.groupby(["model", "layer"]).std(ddof=1)
std[std < 1e-8] = np.nan
std = std.reindex(ind)
std.columns = [c + "_std" for c in std.columns]
df = pd.concat([mean, std], axis=1)
df = df[[mtrc + "_" + st for mtrc in col for st in ["mean", "std"]]]
df.to_csv("../../outputs/results/suppledata/MoA.csv")
df

Bacterial 70S ribosome inhibitor_Balanced Accuracy_mean  \
model                 layer                                                              
Pathological Findings -                                                 0.497962         
Control               Stem                                              0.517367         
                      Block 1                                           0.525001         
                      Block 2                                           0.613446         
                      Block 3                                           0.635521         
...                                                                          ...         
Full                  Block 4                                           0.772522         
                      Block 5                                           0.740049         
                      Block 6                                           0.782984         
                      Block 7                                           0.761306         
                      Head                                              0.537649         

                               Bacterial 70S ribosome inhibitor_Balanced Accuracy_std  \
model                 layer                                                             
Pathological Findings -                                                      NaN        
Control               Stem                                                   NaN        
                      Block 1                                                NaN        
                      Block 2                                                NaN        
                      Block 3                                                NaN        
...                                                                          ...        
Full                  Block 4                                           0.009258        
                      Block 5                                           0.016646        
                      Block 6                                           0.023276        
                      Block 7                                           0.026327        
                      Head                                              0.056355        

                               Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy_mean  \
model                 layer                                                                       
Pathological Findings -                                                 0.500000                  
Control               Stem                                              0.500000                  
                      Block 1                                           0.499284                  
                      Block 2                                           0.591252                  
                      Block 3                                           0.571628                  
...                                                                          ...                  
Full                  Block 4                                           0.593101                  
                      Block 5                                           0.606130                  
                      Block 6                                           0.596273                  
                      Block 7                                           0.581364                  
                      Head                                              0.500060                  

                               Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy_std  \
model                 layer                                                                      
Pathological Findings -                                                      NaN                 
Control               Stem                                                   NaN                 
                      

In [67]:
df.columns

Index(['Bacterial 70S ribosome inhibitor_Balanced Accuracy_mean',
       'Bacterial 70S ribosome inhibitor_Balanced Accuracy_std',
       'Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy_mean',
       'Serotonin 2a (5-HT2a) receptor antagonist_Balanced Accuracy_std',
       'DNA inhibitor_Balanced Accuracy_mean',
       'DNA inhibitor_Balanced Accuracy_std',
       'Cyclooxygenase inhibitor_Balanced Accuracy_mean',
       'Cyclooxygenase inhibitor_Balanced Accuracy_std',
       'Sulfonylurea receptor 1, Kir6.2 blocker_Balanced Accuracy_mean',
       'Sulfonylurea receptor 1, Kir6.2 blocker_Balanced Accuracy_std',
       'Histamine H2 receptor antagonist_Balanced Accuracy_mean',
       'Histamine H2 receptor antagonist_Balanced Accuracy_std',
       'Peroxisome proliferator-activated receptor alpha agonist_Balanced Accuracy_mean',
       'Peroxisome proliferator-activated receptor alpha agonist_Balanced Accuracy_std',
       'Bacterial 70S ribosome inhibitor_AUROC_mean',
   

In [68]:
dfs = []
for seed in range(5):
    df = pd.read_csv(f"../../outputs/230311prognosis_result_{seed+123}.csv")
    df["seed"] = 123 + seed
    df["pathological findings"] = df["finding type"]
    dfs.append(df)
df = pd.concat(dfs)
df.to_csv("../../outputs/results/suppledata/ELPraw.csv", index=False)

In [69]:
df

,model,layer,finding type,AUROC,AP,seed,pathological findings
0,Control,Stem,"Alteration, cytoplasmic",0.793701,0.588567,123,"Alteration, cytoplasmic"
1,Control,Stem,"Alteration, nuclear",0.864156,0.607443,123,"Alteration, nuclear"
2,Control,Stem,Altered hepatocellular foci,0.814909,0.561044,123,Altered hepatocellular foci
3,Control,Stem,Anisonucleosis,0.866892,0.673407,123,Anisonucleosis
4,Control,Stem,Atrophy,0.922763,0.619658,123,Atrophy
...,...,...,...,...,...,...,...
2726,Pathological Findings,-,"Proliferation, bile duct",0.583333,0.582933,127,"Proliferation, bile duct"
2727,Pathological Findings,-,"Proliferation, oval cell",0.500000,0.500000,127,"Proliferation, oval cell"
2728,Pathological Findings,-,Single cell necrosis,0.579815,0.549722,127,Single cell necrosis
2729,Pathological Findings,-,Swelling,0.500000,0.500000,127,Swelling


In [70]:
dft = df.groupby(["model", "layer"]).mean().T

var_df = pd.DataFrame()

for mtrc in ['AUROC', 'AP']:
    vars = []
    for i, m in enumerate(model_name):
        dat = [dft[(m, l)][mtrc] for j, l in enumerate(layer_name)]
        if len(dat) == 1:
            vars.append(np.nan)
        else:
            vars.append(np.var(dat, ddof=0))
    var_df[mtrc] = vars
var_df.index = model_name
var_df
var_df.to_csv("../../outputs/results/230716var_model_prognosis.csv")

In [71]:
var_df = pd.DataFrame()

for mtrc in ['AUROC', 'AP']:
    vars = []
    for i, l in enumerate(layer_name):
        dat = [dft[(m, l)][mtrc] for j, m in enumerate(model_name)]
        if len(dat) == 1:
            vars.append(np.nan)
        else:
            vars.append(np.var(dat, ddof=0))
    var_df[mtrc] = vars
var_df.index = layer_name
var_df
var_df.to_csv("../../outputs/results/230716var_layer_prognosis.csv")

In [72]:
ft_list = df["finding type"].unique()
ind = []
for m in model_name:
    for l in layer_name:
        for p in ft_list:
            ind.append((m,l,p))

In [73]:
mean = df.groupby(["model", "layer", "pathological findings"]).mean().reindex(ind)
del mean["seed"]
mean.columns = [c + "_mean" for c in mean.columns]
std = df.groupby(["model", "layer", "pathological findings"]).std(ddof=1)
std[std < 1e-8] = np.nan
std = std.reindex(ind)
del std["seed"]
std.columns = [c + "_std" for c in std.columns]
df = pd.concat([mean, std], axis=1)
df = df[[mtrc + "_" + st for mtrc in ["AUROC", "AP"] for st in ["mean", "std"]]]
df.to_csv("../../outputs/results/suppledata/ELP.csv")
df

AUROC_mean  AUROC_std   AP_mean  \
model   layer pathological findings                                          
Control Stem  Alteration, cytoplasmic        0.793701        NaN  0.588567   
              Alteration, nuclear            0.864156        NaN  0.607443   
              Altered hepatocellular foci    0.814909        NaN  0.561044   
              Anisonucleosis                 0.866892        NaN  0.673407   
              Atrophy                        0.922763        NaN  0.619658   
...                                               ...        ...       ...   
Full    Head  Proliferation, oval cell       0.958522   0.023230  0.794812   
              Single cell necrosis           0.762923   0.049912  0.670613   
              Swelling                       0.950645   0.014540  0.726290   
              Vacuolization, cytoplasmic     0.721558   0.046876  0.653158   
              DEAD                                NaN        NaN       NaN   

                                             AP_std  
model   layer pathological findings                  
Control Stem  Alteration, cytoplasmic           NaN  
              Alteration, nuclear               NaN  
              Altered hepatocellular foci       NaN  
              Anisonucleosis                    NaN  
              Atrophy                           NaN  
...                                             ...  
Full    Head  Proliferation, oval cell     0.045470  
              Single cell necrosis         0.026060  
              Swelling                     0.057836  
              Vacuolization, cytoplasmic   0.025251  
              DEAD                              NaN  

[2790 rows x 4 columns]

In [74]:
df

AUROC_mean  AUROC_std   AP_mean  \
model   layer pathological findings                                          
Control Stem  Alteration, cytoplasmic        0.793701        NaN  0.588567   
              Alteration, nuclear            0.864156        NaN  0.607443   
              Altered hepatocellular foci    0.814909        NaN  0.561044   
              Anisonucleosis                 0.866892        NaN  0.673407   
              Atrophy                        0.922763        NaN  0.619658   
...                                               ...        ...       ...   
Full    Head  Proliferation, oval cell       0.958522   0.023230  0.794812   
              Single cell necrosis           0.762923   0.049912  0.670613   
              Swelling                       0.950645   0.014540  0.726290   
              Vacuolization, cytoplasmic     0.721558   0.046876  0.653158   
              DEAD                                NaN        NaN       NaN   

                                             AP_std  
model   layer pathological findings                  
Control Stem  Alteration, cytoplasmic           NaN  
              Alteration, nuclear               NaN  
              Altered hepatocellular foci       NaN  
              Anisonucleosis                    NaN  
              Atrophy                           NaN  
...                                             ...  
Full    Head  Proliferation, oval cell     0.045470  
              Single cell necrosis         0.026060  
              Swelling                     0.057836  
              Vacuolization, cytoplasmic   0.025251  
              DEAD                              NaN  

[2790 rows x 4 columns]

In [90]:
dfs = []
for task in ["loo", "MoA", "prognosis"]:
    for mode in ["model", "layer"]:
        df = pd.read_csv(f"../../outputs/results/230716var_{mode}_{task}.csv")
        df = df.rename(columns={"Unnamed: 0":"Layer"})
        df = df.melt(id_vars=df.columns[:1], value_vars=df.columns[1:], var_name="Metric", value_name="Score")
        df["Axis"] = "Frozen" if mode == "model" else "Feature"
        if task=="loo":
            df["Task"] = "LOO" 
        elif task=="MoA":
            df["Task"] = "MoA"
        elif task=="prognosis":
            df["Task"] = "ELP"
        df = df[["Task", "Axis", "Metric", "Layer", "Score"]]
        dfs.append(df)

pd.concat(dfs, axis=0).to_csv("../../outputs/230720curated_var.csv", index=False)

In [89]:
dfs

[   Task    Axis             Metric    Layer     Score
 0   LOO  Frozen              AUROC     Stem  0.000106
 1   LOO  Frozen              AUROC  Block 1  0.000330
 2   LOO  Frozen              AUROC  Block 2  0.000102
 3   LOO  Frozen              AUROC  Block 3  0.000598
 4   LOO  Frozen              AUROC  Block 4  0.000313
 5   LOO  Frozen              AUROC  Block 5  0.000435
 6   LOO  Frozen              AUROC  Block 6  0.000329
 7   LOO  Frozen              AUROC  Block 7  0.000640
 8   LOO  Frozen              AUROC     Head  0.000222
 9   LOO  Frozen                MCC     Stem  0.000393
 10  LOO  Frozen                MCC  Block 1  0.000023
 11  LOO  Frozen                MCC  Block 2  0.000959
 12  LOO  Frozen                MCC  Block 3  0.000375
 13  LOO  Frozen                MCC  Block 4  0.000021
 14  LOO  Frozen                MCC  Block 5  0.000226
 15  LOO  Frozen                MCC  Block 6  0.000403
 16  LOO  Frozen                MCC  Block 7  0.001336
 17  LOO  

In [79]:
df

Unnamed: 0,Unnamed: 0,AUROC,AP
0,Stem,0.000020,0.000011
1,Block 1,0.000016,0.000005
2,Block 2,0.000122,0.000050
3,Block 3,0.000029,0.000003
4,Block 4,0.000143,0.000022
5,Block 5,0.000063,0.000016
6,Block 6,0.000111,0.000064
7,Block 7,0.000511,0.000108
8,Head,0.000376,0.000031
